In [ ]:
import requests
import pandas as pd
from datetime import datetime, timedelta
import io
import os

# Lista de moedas
moedas = ['CAD', 'USD', 'SEK', 'NOK', 'JPY', 'GBP', 'EUR', 'DKK', 'AUD', 'CHF']

# Data final sendo o dia de hoje no formato MM-DD-YYYY
data_final = datetime.today().strftime('%m-%d-%Y')

# Data inicial
data_inicial = '12-30-1999'

# URL base da API
url_base = 'https://olinda.bcb.gov.br/olinda/servico/PTAX/versao/v1/odata/CotacaoMoedaPeriodo'

# Caminho absoluto para a pasta onde os arquivos serão salvos
pasta_destino = r'xxxx'
os.makedirs(pasta_destino, exist_ok=True)

# Função para limpar dados, mantendo apenas o último horário por data
def limpar_dados(file_name):
    if os.path.exists(file_name):
        dados = pd.read_csv(file_name, sep=';')
        if not dados.empty:
            # Convertendo para datetime para facilitar a manipulação
            dados['dataHoraCotacao'] = pd.to_datetime(dados['dataHoraCotacao'])
            dados['data'] = dados['dataHoraCotacao'].dt.date
            # Mantém apenas o último horário disponível para cada data
            dados_limpos = dados.sort_values('dataHoraCotacao').groupby('data').tail(1)
            # Remove a coluna auxiliar 'data'
            dados_limpos = dados_limpos.drop(columns=['data'])
            # Adiciona a coluna 'moeda'
            dados_limpos['moeda'] = dados_limpos['moeda'].fillna(method='ffill')  # Preenche a coluna 'moeda' se necessário
            # Salva os dados limpos de volta ao arquivo CSV
            dados_limpos.to_csv(file_name, sep=';', index=False)
            print(f'Dados limpos no arquivo {file_name}')
        else:
            print(f'O arquivo {file_name} está vazio e não requer limpeza.')
    else:
        print(f'O arquivo {file_name} não existe.')

# Função para baixar e adicionar dados ao arquivo existente
def baixar_dados_incremental(moeda, data_final):
    file_name = os.path.join(pasta_destino, f'{moeda}_cotacoes.csv')
    
    # Se o arquivo já existe, encontrar a última data
    if os.path.exists(file_name):
        # Ler o arquivo existente
        dados_existentes = pd.read_csv(file_name, sep=';')
        if not dados_existentes.empty:
            ultima_data = pd.to_datetime(dados_existentes['dataHoraCotacao'].max())
            data_inicial = (ultima_data + timedelta(days=1)).strftime('%m-%d-%Y')
        else:
            # Se o arquivo existe mas está vazio, use a data inicial original
            data_inicial = '12-30-1999'
    else:
        # Se o arquivo não existe, use a data inicial original
        data_inicial = '12-30-1999'
    
    url = f"{url_base}(moeda=@moeda,dataInicial=@dataInicial,dataFinalCotacao=@dataFinalCotacao)?@moeda='{moeda}'&@dataInicial='{data_inicial}'&@dataFinalCotacao='{data_final}'&$top=1000000&$format=text/csv&$select=paridadeVenda,cotacaoVenda,dataHoraCotacao"
    response = requests.get(url)
    
    if response.status_code == 200:
        # Usa o módulo io para ler o conteúdo CSV da resposta
        novos_dados = pd.read_csv(io.StringIO(response.text))
        if not novos_dados.empty:
            # Adiciona a coluna com o nome da moeda
            novos_dados['moeda'] = moeda
            
            if os.path.exists(file_name):
                # Se o arquivo existe, atualiza com os novos dados
                dados_existentes = pd.read_csv(file_name, sep=';')
                dados_existentes = pd.concat([dados_existentes, novos_dados], ignore_index=True)
                # Salva todos os dados, incluindo os novos e os existentes, com a coluna 'moeda'
                dados_existentes.to_csv(file_name, sep=';', index=False)
            else:
                # Se o arquivo não existe, cria um novo
                novos_dados.to_csv(file_name, sep=';', index=False)
                
            print(f'Dados da moeda {moeda} foram atualizados em {file_name}')
        else:
            print(f'Nenhum dado novo disponível para a moeda {moeda}.')
    else:
        print(f'Falha ao baixar dados para a moeda {moeda}. Status code: {response.status_code}')

    # Limpar dados após adicionar novos dados
    limpar_dados(file_name)

# Baixar dados de forma incremental para cada moeda
for moeda in moedas:
    baixar_dados_incremental(moeda, data_final)
    
    
    




import pandas as pd
from fredapi import Fred

# Substitua 'your_api_key' pela sua chave de API do FRED
api_key = '1bcbbf6cffa7aa308484d98a6078c54d'
fred = Fred(api_key=api_key)

# Definir a data inicial
start_date = '1999-12-30'

# Caminho para o arquivo CSV com as cotações USD/BRL
caminho_arquivo_usd = r'xxxx'

# Função para converter ponto em vírgula e para float
def convert_to_float(value):
    try:
        return float(value.replace(',', '.'))
    except ValueError:
        return value

# Carregar os dados USD/BRL a partir do arquivo CSV
try:
    usd_brl_df = pd.read_csv(caminho_arquivo_usd, sep=';')
    usd_brl_df['dataHoraCotacao'] = pd.to_datetime(usd_brl_df['dataHoraCotacao'])
    usd_brl_df.set_index('dataHoraCotacao', inplace=True)
    
    # Converter a coluna 'cotacaoVenda' de string para float
    usd_brl_df['cotacaoVenda'] = usd_brl_df['cotacaoVenda'].apply(convert_to_float)
    
    # Extrair a última cotação do dia
    usd_brl_df['Data'] = usd_brl_df.index.date
    last_hour_cotations = usd_brl_df.groupby('Data').apply(lambda x: x.iloc[-1])
    last_hour_cotations.set_index('Data', inplace=True)
    
    # Selecionar a coluna 'cotacaoVenda' para a cotação USD/BRL
    usd_brl_last_hour = last_hour_cotations[['cotacaoVenda']]
    usd_brl_last_hour.rename(columns={'cotacaoVenda': 'Cotacao USD/BRL'}, inplace=True)
    
except Exception as e:
    print(f"Erro ao carregar dados do arquivo CSV: {e}")
    usd_brl_last_hour = pd.DataFrame(columns=['Cotacao USD/BRL'])

# Obter dados de taxas de câmbio USD/CNY do FRED
try:
    usd_cny = fred.get_series('DEXCHUS', start_date)
    
    # Criar DataFrame para USD/CNY
    usd_cny_df = pd.DataFrame({
        'Data': usd_cny.index,
        'Cotacao USD/CNY': usd_cny.values
    })
    usd_cny_df.set_index('Data', inplace=True)
    
    # Reindexar o DataFrame USD/CNY para combinar com o DataFrame USD/BRL
    combined_df = usd_cny_df.reindex(usd_brl_last_hour.index).fillna(method='ffill')
    
    # Calcular a paridade CNY/USD
    combined_df['Paridade CNY/USD'] = 1 / combined_df['Cotacao USD/CNY']
    
    # Adicionar a cotação USD/BRL ao DataFrame combinado
    final_df = combined_df.join(usd_brl_last_hour)
    
    # Calcular a cotação CNY/BRL
    final_df['Cotacao CNY/BRL'] = final_df['Cotacao USD/BRL'] / final_df['Cotacao USD/CNY']
    
except Exception as e:
    print(f"Erro ao obter dados do FRED: {e}")
    final_df = pd.DataFrame(columns=['Paridade CNY/USD', 'Cotacao USD/BRL', 'Cotacao CNY/BRL'])

# Remover colunas desnecessárias e renomear as colunas restantes
final_df.drop(columns=['Cotacao USD/CNY', 'Cotacao USD/BRL'], inplace=True, errors='ignore')
final_df.rename(columns={
    'Paridade CNY/USD': 'paridadeVenda',
    'Cotacao CNY/BRL': 'cotacaoVenda'
}, inplace=True)
final_df.index.name = 'dataHoraCotacao'

# Adicionar a coluna 'moeda' com o valor 'CNY'
final_df['moeda'] = 'CNY'

# Exibir o DataFrame resultante
print(final_df.head())

# Exibir os valores da última data para depuração
if not final_df.empty:
    last_date = final_df.index[-1]
    print(f"\nValores para a última data ({last_date}):")
    print(f"  Cotacao CNY/BRL: {final_df.loc[last_date, 'cotacaoVenda']}")
    print(f"  Paridade CNY/USD: {final_df.loc[last_date, 'paridadeVenda']}")

# Função para formatar valores com vírgulas, mantendo todas as casas decimais
def format_with_comma(value):
    try:
        # Converte o valor para string e substitui o ponto por vírgula
        return str(value).replace('.', ',')
    except ValueError:
        return value

# Aplicar a formatação com vírgula
final_df_formatted = final_df.applymap(format_with_comma)
final_df_formatted.index.name = 'dataHoraCotacao'

# Caminho para salvar o arquivo CSV
caminho_arquivo_final = r'xxxx'

# Salvar o DataFrame em um arquivo CSV separado por ponto e vírgula
final_df_formatted.to_csv(caminho_arquivo_final, sep=';', index=True)

import pandas as pd
import os

# Caminho absoluto para o arquivo CSV consolidado
caminho_arquivo_consolidado = r'xxxx'

# Verifique se o arquivo existe e apague-o
if os.path.exists(caminho_arquivo_consolidado):
    os.remove(caminho_arquivo_consolidado)
    print(f'O arquivo {caminho_arquivo_consolidado} foi apagado.')
else:
    print(f'O arquivo {caminho_arquivo_consolidado} não existe.')

# Caminho absoluto para a pasta onde os arquivos estão salvos
pasta_destino = r'xxxx'

# Caminho para o arquivo CSV consolidado
caminho_arquivo_consolidado = os.path.join(pasta_destino, 'todas_as_moedas_cotacoes.csv')

# Listar todos os arquivos CSV na pasta
arquivos_csv = [f for f in os.listdir(pasta_destino) if f.endswith('_cotacoes.csv')]

# Inicializar uma lista para armazenar os DataFrames
lista_dfs = []

# Ler e adicionar todos os arquivos CSV à lista de DataFrames
for arquivo_csv in arquivos_csv:
    caminho_arquivo = os.path.join(pasta_destino, arquivo_csv)
    try:
        # Ler o arquivo CSV
        df = pd.read_csv(caminho_arquivo, sep=';')
        
        # Converter a coluna 'dataHoraCotacao' para o formato de data
        if 'dataHoraCotacao' in df.columns:
            df['dataHoraCotacao'] = pd.to_datetime(df['dataHoraCotacao'], errors='coerce')
            df['dataHoraCotacao'] = df['dataHoraCotacao'].dt.strftime('%d/%m/%Y')
        
        # Garantir que 'paridadeVenda' e 'cotacaoVenda' sejam numéricos
        for coluna in ['paridadeVenda', 'cotacaoVenda']:
            if coluna in df.columns:
                # Substituir separador decimal por ponto para conversão
                df[coluna] = df[coluna].astype(str).str.replace(',', '.', regex=False)
                # Converter para número, definindo valores inválidos como NaN
                df[coluna] = pd.to_numeric(df[coluna], errors='coerce')
                # Reverter o separador decimal para vírgula após a conversão
                df[coluna] = df[coluna].map(lambda x: f"{x:,.6f}".replace('.', ',') if pd.notna(x) else x)

        # Adicionar a coluna 'moeda'
        if 'moeda' not in df.columns:
            df['moeda'] = arquivo_csv.split('_')[0]

        lista_dfs.append(df)
        print(f'{arquivo_csv} lido e formatado com sucesso.')
    except Exception as e:
        print(f'Erro ao ler {arquivo_csv}: {e}')

# Concatenar todos os DataFrames em um único DataFrame
if lista_dfs:
    df_consolidado = pd.concat(lista_dfs, ignore_index=True)

    # Salvar o DataFrame consolidado em um arquivo CSV separado por ponto e vírgula
    df_consolidado.to_csv(caminho_arquivo_consolidado, sep=';', index=False, float_format='%.6f')
    print(f'Arquivo consolidado criado com sucesso: {caminho_arquivo_consolidado}')
else:
    print('Nenhum arquivo CSV encontrado para consolidar.')

import pandas as pd
from datetime import datetime

# Caminho para o arquivo CSV
caminho_arquivo = r'xxxx'

# Ler o arquivo CSV
df = pd.read_csv(caminho_arquivo, sep=';')

# Converter a coluna 'dataHoraCotacao' para datetime
df['dataHoraCotacao'] = pd.to_datetime(df['dataHoraCotacao'], format='%d/%m/%Y')

# Criar um DataFrame com todas as datas entre 30/12/1999 e hoje
data_inicial = datetime(1999, 12, 30)
data_atual = datetime.now()
datas = pd.date_range(start=data_inicial, end=data_atual)

# Criar um DataFrame vazio com as datas e moedas
moedas = df['moeda'].unique()
dados_completos = pd.MultiIndex.from_product([datas, moedas], names=['dataHoraCotacao', 'moeda'])
df_completo = pd.DataFrame(index=dados_completos).reset_index()

# Merge para preencher os valores
df_completo = pd.merge(df_completo, df, on=['dataHoraCotacao', 'moeda'], how='left')

# Preencher valores faltantes
df_completo.sort_values(by=['moeda', 'dataHoraCotacao'], inplace=True)
df_completo['paridadeVenda'].fillna(method='ffill', inplace=True)
df_completo['cotacaoVenda'].fillna(method='ffill', inplace=True)

# Filtrar para datas a partir de 01/01/2000
data_filtro = datetime(2000, 1, 1)
df_completo = df_completo[df_completo['dataHoraCotacao'] >= data_filtro]

# Adicionar coluna com a última data disponível para cada moeda
ultima_data_disponivel = df.groupby('moeda')['dataHoraCotacao'].max().reset_index()
ultima_data_disponivel.columns = ['moeda', 'ultimaDataDisponivel']

# Merge para adicionar a coluna de última data disponível
df_completo = pd.merge(df_completo, ultima_data_disponivel, on='moeda', how='left')

# Adicionar coluna com a data da última execução do script
data_execucao = datetime.now().strftime('%d/%m/%Y %H:%M:%S')
df_completo['dataExecucao'] = data_execucao

# Sobrescrever o arquivo CSV original
df_completo.to_csv(caminho_arquivo, sep=';', index=False)

print(f"Arquivo original sobrescrito com dados a partir de 01/01/2000: {caminho_arquivo}")

import pandas as pd

# Carregar o arquivo CSV
file_path = r"xxxx"
df = pd.read_csv(file_path, sep=';', decimal=',')

# Verificar as colunas disponíveis
print(df.columns)

# Converter colunas para numérico
df['cotacaoVenda'] = pd.to_numeric(df['cotacaoVenda'], errors='coerce')
df['paridadeVenda'] = pd.to_numeric(df['paridadeVenda'], errors='coerce')

# Preencher valores ausentes usando ffill
df['cotacaoVenda'].ffill(inplace=True)

# Inspecionar o formato das datas no DataFrame
print(df[['dataExecucao', 'ultimaDataDisponivel']].head())

# Tentar converter as datas com um formato flexível
df['dataExecucao'] = pd.to_datetime(df['dataExecucao'], errors='coerce', infer_datetime_format=True)
df['ultimaDataDisponivel'] = pd.to_datetime(df['ultimaDataDisponivel'], errors='coerce', infer_datetime_format=True)

# Verificar se as datas foram convertidas corretamente
print(df[['dataExecucao', 'ultimaDataDisponivel']].head())

# Tabela Cruzada de Cotações (em BRL)
tabela_cotacao = df.pivot_table(index='dataHoraCotacao', columns='moeda', values='cotacaoVenda')
# Garantir que o BRL esteja incluído
tabela_cotacao['BRL'] = 1

# Resetar o índice para 'dataHoraCotacao'
tabela_cotacao.reset_index(inplace=True)

# Tabela Cruzada de Paridades
# Criar DataFrame para cotações em BRL
cotacoes_brl = df.pivot_table(index='dataHoraCotacao', columns='moeda', values='cotacaoVenda', aggfunc='mean')
cotacoes_brl['BRL'] = 1

# Inicializar DataFrame para paridades
paridades = pd.DataFrame(index=cotacoes_brl.index)

# Calcular paridades
moedas = cotacoes_brl.columns
for moeda1 in moedas:
    for moeda2 in moedas:
        if moeda1 != moeda2:
            paridade_col = f'{moeda1}/{moeda2}'
            paridades[paridade_col] = cotacoes_brl[moeda1] / cotacoes_brl[moeda2]

# Resetar os índices
paridades.reset_index(inplace=True)
tabela_cotacao.reset_index(inplace=True)

# Adicionar colunas de dataExecucao e ultimaDataDisponivel nas tabelas
if 'dataExecucao' in df.columns and 'ultimaDataDisponivel' in df.columns:
    tabela_cotacao = pd.merge(tabela_cotacao, df[['dataHoraCotacao', 'dataExecucao', 'ultimaDataDisponivel']].drop_duplicates(), on='dataHoraCotacao', how='left')
    paridades = pd.merge(paridades, df[['dataHoraCotacao', 'dataExecucao', 'ultimaDataDisponivel']].drop_duplicates(), on='dataHoraCotacao', how='left')

# Função para formatar valores numéricos com 4 casas decimais
def format_decimal(df):
    df = df.copy()  # Evitar problemas com fragmentação
    for col in df.columns:
        if pd.api.types.is_numeric_dtype(df[col]):
            df[col] = df[col].apply(lambda x: '{:,.4f}'.format(x).replace('.', ',') if pd.notna(x) else '')
    return df

# Aplicar a formatação
tabela_cotacao_formatada = format_decimal(tabela_cotacao)
paridades_formatada = format_decimal(paridades)

# Salvar os DataFrames em arquivos CSV
tabela_cotacao_formatada.to_csv(r"xxxx", sep=';', index=False)
paridades_formatada.to_csv(r"xxxx", sep=';', index=False)


import pandas as pd

# Caminho do arquivo CSV original
file_path = r"xxxx"

# Ler o arquivo CSV com separador ";"
df = pd.read_csv(file_path, sep=";")

# Selecionar as colunas necessárias
colunas_desejadas = [
    "dataHoraCotacao", "EUR/BRL", "EUR/GBP", "EUR/USD", "BRL/EUR", "BRL/GBP", "BRL/USD",
    "GBP/BRL", "GBP/EUR", "GBP/USD", "USD/BRL", "USD/GBP", "USD/EUR", "ultimaDataDisponivel"
]

df_selecionado = df[colunas_desejadas]

# Desempilhar os dados de câmbio para um formato longo
df_transformado = df_selecionado.melt(
    id_vars=["dataHoraCotacao", "ultimaDataDisponivel"], 
    var_name="Paridade", 
    value_name="Valor"
)

# Encontrar os valores das últimas datas para cada paridade
df_ultima_data = df_transformado[df_transformado['dataHoraCotacao'] == df_transformado['ultimaDataDisponivel']]

# Realizar o merge para adicionar a coluna Valor-1
df_resultado = pd.merge(df_transformado, df_ultima_data[['Paridade', 'Valor']], 
                        on='Paridade', suffixes=('', '-1'), how='left')

# Limpeza: substituir vírgulas por pontos, remover espaços e garantir que os valores sejam numéricos
df_resultado['Valor'] = df_resultado['Valor'].replace({',': '.'}, regex=True)  # Substitui vírgulas por pontos
df_resultado['Valor-1'] = df_resultado['Valor-1'].replace({',': '.'}, regex=True)  # Substitui vírgulas por pontos

# Remover espaços extras (se existirem)
df_resultado['Valor'] = df_resultado['Valor'].str.strip()
df_resultado['Valor-1'] = df_resultado['Valor-1'].str.strip()

# Converter para valores numéricos (float), forçando a conversão correta
df_resultado['Valor'] = pd.to_numeric(df_resultado['Valor'], errors='coerce')
df_resultado['Valor-1'] = pd.to_numeric(df_resultado['Valor-1'], errors='coerce')

# Calcular a variação percentual entre "Valor" (mais antigo) e "Valor-1" (mais recente)
df_resultado['Variação Percentual'] = ((df_resultado['Valor'] ) / df_resultado['Valor-1'])

# Converter os valores numéricos das colunas 'Valor', 'Valor-1' e 'Variação Percentual' para string e substituir o ponto por vírgula
df_resultado['Valor'] = df_resultado['Valor'].apply(lambda x: str(x).replace('.', ',') if pd.notna(x) else x)
df_resultado['Valor-1'] = df_resultado['Valor-1'].apply(lambda x: str(x).replace('.', ',') if pd.notna(x) else x)
df_resultado['Variação Percentual'] = df_resultado['Variação Percentual'].apply(lambda x: str(x).replace('.', ',') if pd.notna(x) else x)


# Alterar os nomes das colunas
df_resultado.rename(columns={
    'dataHoraCotacao': 'Data da paridade',
    'ultimaDataDisponivel': 'Ultima Data Disponivel',
    'Variação Percentual': '%'
}, inplace=True)

# Exibir o resultado para verificar
print(df_resultado.head())

# Salvar o DataFrame resultante em um novo arquivo CSV
output_path = r"xxxx"
df_resultado.to_csv(output_path, sep=";", index=False)


PermissionError: [WinError 5] Acesso negado: 'C:\\Users\\DXX7'